In [15]:
from api import static, gameweek, fixtures

import pandas as pd
import numpy as np
import sqlalchemy as sa

In [3]:
from setup import setup
engine = setup()

In [4]:
fix = fixtures().fixtures
static_info = static()
teams = static_info.get_teams_info()
events = static_info.get_events()

In [9]:
fix['stats'].astype(str)

0                                                     []
1      [{'identifier': 'goals_scored', 'a': [{'value'...
2      [{'identifier': 'goals_scored', 'a': [{'value'...
3      [{'identifier': 'goals_scored', 'a': [{'value'...
4      [{'identifier': 'goals_scored', 'a': [{'value'...
                             ...                        
375                                                   []
376                                                   []
377                                                   []
378                                                   []
379                                                   []
Name: stats, Length: 380, dtype: object

In [19]:
# with engine.connect() as conn:
#     try:
#         fix['stats'] = fix['stats'].astype(str)
#         fix.to_sql('fixtures', con=conn,index=False, if_exists='replace')
#         conn.commit()
#     except Exception as e:
#         conn.rollback()
#         raise e
#     finally:
#         conn.close()

In [17]:
# with engine.connect() as conn:
#     try:
#         result = conn.execute(sa.text("SELECT * FROM teams_info;"))
#         rows = result.fetchall()
#         print("Result set:", rows)
#         conn.commit()
#     except Exception as e:
#         conn.rollback()
#         raise e
#     finally:
#         conn.close()

Result set: [('3', 0, None, '1', 0, 'Arsenal', 0, 0, 0, 'ARS', 4, None, 0, 0, 1230, 1285, 1250, 1250, 1210, 1320, 1), ('7', 0, None, '2', 0, 'Aston Villa', 0, 0, 0, 'AVL', 3, None, 0, 0, 1115, 1175, 1130, 1190, 1100, 1160, 2), ('91', 0, None, '3', 0, 'Bournemouth', 0, 0, 0, 'BOU', 3, None, 0, 0, 1060, 1095, 1050, 1100, 1060, 1090, 127), ('94', 0, None, '4', 0, 'Brentford', 0, 0, 0, 'BRE', 3, None, 0, 0, 1125, 1205, 1120, 1220, 1130, 1190, 130), ('36', 0, None, '5', 0, 'Brighton', 0, 0, 0, 'BHA', 3, None, 0, 0, 1165, 1210, 1120, 1200, 1210, 1240, 131), ('90', 0, None, '6', 0, 'Burnley', 0, 0, 0, 'BUR', 2, None, 0, 0, 1060, 1080, 1060, 1080, 1060, 1080, 43), ('8', 0, None, '7', 0, 'Chelsea', 0, 0, 0, 'CHE', 3, None, 0, 0, 1115, 1160, 1130, 1210, 1100, 1110, 4), ('31', 0, None, '8', 0, 'Crystal Palace', 0, 0, 0, 'CRY', 3, None, 0, 0, 1100, 1100, 1140, 1170, 1080, 1085, 6), ('11', 0, None, '9', 0, 'Everton', 0, 0, 0, 'EVE', 3, None, 0, 0, 1075, 1100, 1070, 1120, 1080, 1080, 7), ('54', 0, N